<a href="https://colab.research.google.com/github/probabll/mixed-rv-vae/blob/master/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

On Colab, you will need to clone and install [probabll/dists.pt](https://github.com/probabll/dists.pt.git)

In [2]:
import torch
torch.__version__

'1.8.1+cu102'

In [3]:
import numpy as np
import torch
import torch.distributions as td
import probabll.distributions as pd
import matplotlib.pyplot as plt
import torch.nn as nn

In [4]:
from collections import namedtuple, OrderedDict, defaultdict
from tqdm.auto import tqdm
from itertools import chain
from tabulate import tabulate

In [5]:
from components import GenerativeModel, InferenceModel, VAE
from data import load_mnist
from main import load_cfg, make_args, make_state, get_batcher, validate

In [6]:
from analysis import compare_marginals, compare_samples

# Load model and data

* Load hyperparameters
* Load model state
* Load MNIST data

In [7]:
files = {
    'gaussian': [
        'neurips-mixed-rv/submission/likely-oath-1/',
        'neurips-mixed-rv/submission/swept-waterfall-2/',
        'neurips-mixed-rv/submission/astral-deluge-3/',
    ],
    'gaussian9': [
        'neurips-mixed-rv/submission/light-darkness-23',
        'neurips-mixed-rv/submission/eternal-dragon-24',
        'neurips-mixed-rv/submission/northern-terrain-25',
    ],
    'dirichlet-1': [
        'neurips-mixed-rv/submission/giddy-breeze-4/',
        'neurips-mixed-rv/submission/feasible-sunset-5/',
        'neurips-mixed-rv/submission/snowy-darkness-6/',
    ],
    'dirichlet-0.5': [
        'neurips-mixed-rv/submission/toasty-morning-7',
        'neurips-mixed-rv/submission/stoic-firebrand-8',
        'neurips-mixed-rv/submission/woven-cosmos-9',
    ],
    'maxent-0': [        
        'neurips-mixed-rv/IvI/valiant-mountain-5/',
        'neurips-mixed-rv/IvI/young-totem-6/',
        'neurips-mixed-rv/IvI/youthful-pond-7/',
        #'neurips-mixed-rv/IvI/fresh-energy-8/',
        #'neurips-mixed-rv/IvI/sage-haze-105/',
    ],
    'categorical': [
        'neurips-mixed-rv/submission/upbeat-star-31/'
    ],
    'categorical2': [
        'neurips-mixed-rv/submission/electric-pyramid-29',
        'neurips-mixed-rv/submission/upbeat-star-31/'
    ],
    'mixedrv-sharednet': [
        'neurips-mixed-rv/submission/deft-tree-32',  # figures in paper
        'neurips-mixed-rv/submission/icy-sponge-36'
    ]
}

In [8]:
def test(runs, device, num_samples=1000):
    header = ['ELBO', 'D', 'R', 'R_F', 'R_Y', 'NLL', 'BPD']

    metrics = []

    for path in runs:
        print("Loading args")
        args = make_args(
            load_cfg(
                f"{path}/cfg.json", 
                # use this to specify a decide for analysis
                device=device,
                # use this to change paths if you need
                data_dir='./tmp',
                # you don't really need to change the output_dir
            )
        )
        print(args)
        print(f"Loading model: {path}")
        state = make_state(
            args, 
            device=args.device, 
            # change this to .best if you want the best ckpt (rather than the last)
            ckpt_path=f"{path}/ckpt.best"
        )
        print("Loading data")
        train_loader, valid_loader, test_loader = load_mnist(
            args.batch_size, 
            save_to=args.data_dir, 
            height=args.height, 
            width=args.width
        )
        print("Evaluating")
        test_metrics = validate(state.vae, get_batcher(test_loader, args), num_samples, compute_DR=True)
        print(f'Saved ckpt - Test: nll={test_metrics[0]:.2f} bpd={test_metrics[1]:.2f}')
        rows = [('IS-NLL', test_metrics[0], None), ('IS-BPD', test_metrics[1], None)]

        metrics.append([])
        print(test_metrics[2].keys())
        for k in ['ELBO', 'D', 'R', 'R_F', 'R_Y']:
            if k in test_metrics[2]:
                v = test_metrics[2][k]
                rows.append((k, v.mean(), v.std()))
                metrics[-1].append(v.mean())
            else:
                metrics[-1].append(-np.inf)

        metrics[-1].append(test_metrics[0].item())
        metrics[-1].append(test_metrics[1].item())
        
        print(tabulate(rows, headers=['metric', 'mean', 'std']))    
        
    print("\n All runs\n")
    print(tabulate(metrics, headers=header))

    print("\n\nAverage\n")
    M = np.array(metrics).mean(0, keepdims=True)
    M = np.where(M == -np.inf, None, M)
    print(tabulate(M, headers=header, floatfmt=".2f", tablefmt='latex', missingval='-'))
    print()
    return metrics, header

# Gaussian

In [101]:
gaussian_m, gaussian_h = test(files['gaussian'], 'cuda:1')

Loading args
Setting wandb to default False
Setting wandb_watch to default False
Overriding device to user choice cuda:0
Overriding data_dir to user choice ./tmp
Config(seed=10, batch_size=200, data_dir='./tmp', height=28, width=28, output_dir='neurips-mixed-rv/submission/likely-oath-1', device='cuda:0', y_dim=0, z_dim=10, prior_z='gaussian 0.0 1.0', hidden_dec_size=500, posterior_z='gaussian', hidden_enc_size=500, epochs=200, num_samples=100, gen_opt='adam', gen_lr=0.001, gen_l2=0.0, gen_p_drop=0.0, inf_opt='adam', inf_lr=0.001, inf_l2=1e-06, inf_p_drop=0.0, grad_clip=5.0, project='neurips21-submission', cfg='cfg/gaussian-vae.json', prior_f='gibbs 0.0', prior_y='dirichlet 1.0', posterior_f='gibbs -10 10', posterior_y='dirichlet 1e-3 1e3', shared_concentrations=True, mean_field=True, training_samples=1, load_ckpt=False, reset_opt=False, exact_marginal=False, use_self_critic=False, use_reward_standardisation=False, tqdm=False, wandb=False, wandb_watch=False)
Loading model: neurips-mixed

# Gaussian K=9

In [10]:
gaussian9_m, gaussian9_h = test(files['gaussian9'], 'cuda:1')

Loading args
Setting wandb to default False
Setting wandb_watch to default False
Overriding device to user choice cuda:1
Overriding data_dir to user choice ./tmp
Config(seed=10, batch_size=200, data_dir='./tmp', height=28, width=28, output_dir='neurips-mixed-rv/submission/light-darkness-23', device='cuda:1', y_dim=0, z_dim=9, prior_z='gaussian 0.0 1.0', hidden_dec_size=500, posterior_z='gaussian', hidden_enc_size=500, epochs=500, num_samples=100, gen_opt='adam', gen_lr=0.001, gen_l2=0.0, gen_p_drop=0.0, inf_opt='adam', inf_lr=0.001, inf_l2=1e-06, inf_p_drop=0.0, grad_clip=5.0, project='neurips21-submission', cfg='cfg/gaussian-vae.json', prior_f='gibbs 0.0', prior_y='dirichlet 1.0', posterior_f='gibbs -10 10', posterior_y='dirichlet 1e-3 1e3', shared_concentrations=True, share_fy_net=False, mean_field=True, training_samples=1, load_ckpt=None, reset_opt=False, exact_marginal=False, use_self_critic=False, use_reward_standardisation=False, tqdm=False, wandb=False, wandb_watch=False)
Loadin

# Dirichlet-1

In [100]:
dir1_m, dir1_h = test(files['dirichlet-1'], 'cuda:1')

Loading args
Setting wandb to default False
Setting wandb_watch to default False
Overriding device to user choice cuda:0
Overriding data_dir to user choice ./tmp
Config(seed=10, batch_size=200, data_dir='./tmp', height=28, width=28, output_dir='neurips-mixed-rv/submission/giddy-breeze-4', device='cuda:0', y_dim=0, z_dim=10, prior_z='dirichlet 1.0', hidden_dec_size=500, posterior_z='dirichlet 1e-3 1e3', hidden_enc_size=500, epochs=200, num_samples=100, gen_opt='adam', gen_lr=0.001, gen_l2=0.0, gen_p_drop=0.0, inf_opt='adam', inf_lr=0.001, inf_p_drop=0.0, grad_clip=5.0, project='neurips21-submission', cfg='cfg/dirichlet-vae.json', prior_f='gibbs 0.0', prior_y='dirichlet 1.0', posterior_f='gibbs -10 10', posterior_y='dirichlet 1e-3 1e3', shared_concentrations=True, mean_field=True, training_samples=1, inf_l2=0.0, load_ckpt=False, reset_opt=False, exact_marginal=False, use_self_critic=False, use_reward_standardisation=False, tqdm=False, wandb=False, wandb_watch=False)
Loading model: neurip

# Dirichlet 0.5

In [10]:
dir05_m, dir05_h = test(files['dirichlet-0.5'], 'cuda:1')

Loading args
Setting wandb to default False
Setting wandb_watch to default False
Setting share_fy_net to default False
Overriding device to user choice cuda:0
Overriding data_dir to user choice ./tmp
Config(seed=10, batch_size=200, data_dir='./tmp', height=28, width=28, output_dir='neurips-mixed-rv/submission/toasty-morning-7', device='cuda:0', y_dim=0, z_dim=10, prior_z='dirichlet 0.5', hidden_dec_size=500, posterior_z='dirichlet 1e-3 1e3', hidden_enc_size=500, epochs=200, num_samples=100, gen_opt='adam', gen_lr=0.001, gen_l2=0.0, gen_p_drop=0.0, inf_opt='adam', inf_lr=0.001, inf_p_drop=0.0, grad_clip=5.0, project='neurips21-submission', cfg='cfg/dirichlet-vae.json', prior_f='gibbs 0.0', prior_y='dirichlet 1.0', posterior_f='gibbs -10 10', posterior_y='dirichlet 1e-3 1e3', shared_concentrations=True, mean_field=True, training_samples=1, inf_l2=0.0, load_ckpt=False, reset_opt=False, exact_marginal=False, use_self_critic=False, use_reward_standardisation=False, tqdm=False, wandb=False, 

# MaxEnt 0

In [12]:
maxent0_m, maxent0_h = test(files['maxent-0'], 'cuda:1')

Loading args
Setting project to default neurips21
Setting cfg to default None
Setting share_fy_net to default False
Overriding device to user choice cuda:0
Overriding data_dir to user choice ./tmp
Config(seed=14, batch_size=200, data_dir='./tmp', height=28, width=28, output_dir='/ssdstore/vnicula/neurips-mixed-rv/IvI/mixed-vae/valiant-mountain-5', device='cuda:0', y_dim=10, prior_f='gibbs-max-ent 0', prior_y='dirichlet 1.0', z_dim=0, hidden_dec_size=500, posterior_f='gibbs -10 10', posterior_y='dirichlet 1e-3 1e3', shared_concentrations=True, mean_field=True, hidden_enc_size=500, epochs=500, num_samples=100, gen_opt='adam', gen_lr=0.001, gen_l2=0.0, gen_p_drop=0.0, inf_opt='adam', inf_lr=0.0005, inf_l2=1e-06, inf_p_drop=0.1, grad_clip=5.0, training_samples=1, exact_marginal=False, use_self_critic=True, use_reward_standardisation=False, wandb=False, wandb_watch=False, tqdm=False, prior_z='gaussian 0.0 1.0', posterior_z='gaussian', load_ckpt=False, reset_opt=False, project='neurips21', c

In [9]:
maxent0shared_m, maxent0shared_h = test(files['mixedrv-sharednet'], 'cuda:1')

Loading args
Setting wandb to default False
Setting wandb_watch to default False
Overriding device to user choice cuda:1
Overriding data_dir to user choice ./tmp
Config(seed=10, batch_size=100, data_dir='./tmp', height=28, width=28, output_dir='neurips-mixed-rv/submission/deft-tree-32', device='cuda:1', y_dim=10, prior_f='gibbs-max-ent 0', prior_y='dirichlet 1.0', z_dim=0, hidden_dec_size=500, posterior_f='gibbs -10 10', posterior_y='dirichlet 1e-3 1e3', shared_concentrations=True, mean_field=True, hidden_enc_size=500, epochs=500, num_samples=100, gen_opt='adam', gen_lr=0.0005, gen_l2=0.0, gen_p_drop=0.0, inf_opt='adam', inf_lr=0.0005, inf_l2=1e-06, inf_p_drop=0.1, grad_clip=1.0, use_self_critic=0, use_reward_standardisation=1, project='neurips21-submission', cfg='cfg/mixed-vae.json', prior_z='gaussian 0.0 1.0', posterior_z='gaussian', share_fy_net=1, training_samples=1, load_ckpt=None, reset_opt=False, exact_marginal=False, tqdm=False, wandb=False, wandb_watch=False)
Loading model: ne

# Categorical (Exact Gradient)

In [9]:
cat_m, cat_h = test(files['categorical'], 'cuda:1')

Loading args
Setting wandb to default False
Setting wandb_watch to default False
Overriding device to user choice cuda:1
Overriding data_dir to user choice ./tmp
Config(seed=10, batch_size=200, data_dir='./tmp', height=28, width=28, output_dir='neurips-mixed-rv/submission/upbeat-star-31', device='cuda:1', y_dim=10, prior_f='categorical 0.0', prior_y='identity', z_dim=0, hidden_dec_size=500, posterior_f='categorical -10 10', posterior_y='identity', shared_concentrations=True, mean_field=True, hidden_enc_size=500, epochs=500, num_samples=100, gen_opt='adam', gen_lr=0.001, gen_l2=0.0, gen_p_drop=0.0, inf_opt='adam', inf_lr=0.0005, inf_l2=1e-06, inf_p_drop=0.1, grad_clip=1.0, use_self_critic=1, use_reward_standardisation=0, project='neurips21-submission', cfg='cfg/mixed-vae.json', prior_z='gaussian 0.0 1.0', posterior_z='gaussian', share_fy_net=False, training_samples=1, load_ckpt=None, reset_opt=False, exact_marginal=1, tqdm=False, wandb=False, wandb_watch=False)
Loading model: neurips-mi

In [9]:
_ = test(files['categorical2'], 'cuda:1')

Loading args
Setting wandb to default False
Setting wandb_watch to default False
Overriding device to user choice cuda:1
Overriding data_dir to user choice ./tmp
Config(seed=10, batch_size=200, data_dir='./tmp', height=28, width=28, output_dir='neurips-mixed-rv/submission/electric-pyramid-29', device='cuda:1', y_dim=10, prior_f='categorical 0.0', prior_y='identity', z_dim=0, hidden_dec_size=500, posterior_f='categorical -10 10', posterior_y='identity', shared_concentrations=True, mean_field=True, hidden_enc_size=500, epochs=500, num_samples=100, gen_opt='adam', gen_lr=0.001, gen_l2=0.0, gen_p_drop=0.0, inf_opt='adam', inf_lr=0.0005, inf_l2=1e-06, inf_p_drop=0.1, grad_clip=1.0, use_self_critic=1, use_reward_standardisation=0, project='neurips21-submission', cfg='cfg/mixed-vae.json', prior_z='gaussian 0.0 1.0', posterior_z='gaussian', share_fy_net=False, training_samples=1, load_ckpt=None, reset_opt=False, exact_marginal=False, tqdm=False, wandb=False, wandb_watch=False)
Loading model: n